In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import cv2 as cv
import json
import os
import pandas as pd
from tqdm import tqdm
import operator

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)
import matplotlib.image as mpimg

import geojson
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from Utils.Postprocessing.post_process import postprocess_mask, create_geojson

import geopandas as gpd
import rtree

In [ ]:
def find_distance(gdf_endocard):
    idx = rtree.index.Index()
    for i, row in gdf_endocard.iterrows():
        idx.insert(i, row.geometry.bounds)

    for i, row in gdf_endocard.iterrows():
        for j in idx.intersection(row.geometry.buffer(15).bounds):
            if i < j and row.geometry.distance(gdf_endocard.loc[j, 'geometry']) < 15:
                return True

    return False


def merge_nearest_endocards(mask):
    endocard_mask = mask[:,:,-1]
    struct_elem = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11, 11))
    dilate_count = 0

    while True:
        geojson_file = create_geojson(endocard_mask, [
            "endocariums"
        ])
        gdf_endocard = gpd.GeoDataFrame.from_features(geojson_file)

        distance = find_distance(gdf_endocard)
        if distance:
            endocard_mask = cv.dilate(endocard_mask, struct_elem)
            dilate_count += 1
        else:
            break

    for _ in range(dilate_count):
        endocard_mask = cv.erode(endocard_mask, struct_elem)

    mask[:,:,-1] = endocard_mask
    return mask

In [ ]:
def IoU(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    intersect = np.sum(y_true * y_pred)
    fsum = np.sum(y_true)
    ssum = np.sum(y_pred)
    dice = (2 * intersect ) / (fsum + ssum)
    return dice

In [ ]:
def convert_to_multiclass(mask, num_classes):
    new_mask = np.zeros((mask.shape[0], mask.shape[1], num_classes), dtype=np.uint8)
    
    new_mask[:, :, 4] = mask[:, :, 0]
    new_mask[:, :, 5] = mask[:, :, 1]
    new_mask[:, :, 6] = mask[:, :, 2]
    
    new_mask[:, :, 1] = cv.bitwise_and(new_mask[:, :, 4], new_mask[:, :, 5])    # blood + infla
    new_mask[:, :, 2] = cv.bitwise_and(new_mask[:, :, 6], new_mask[:, :, 5])    # endocard + infla
    new_mask[:, :, 3] = cv.bitwise_and(new_mask[:, :, 4], new_mask[:, :, 6])    # blood + endocard
    
    union = new_mask[:, :, 0]
    for i in range(1, num_classes):
        union = cv.bitwise_or(union, new_mask[:, :, i])
    union = np.clip(union, 0, 1)
    new_mask[:, :, 0] = np.where((union == 0) | (union == 1), union ^ 1, union)

    return new_mask

In [ ]:
def get_mask_index(feature, classes):
    class_type = feature['properties']['classification']['name']

    for idx, name in enumerate(classes):
        if class_type.lower() == name.lower():
            return idx

    # else return Other cells
    return 0


def get_mask(shape, annotations, classes):
    x, y = int(shape[0]), int(shape[1])

    classes_masks = [
        np.zeros((x, y, 1), dtype='uint8')
        for _ in range(len(classes))
    ]

    for feat in annotations:
        geometry_name = 'geometry'
        coors = feat[geometry_name]['coordinates'][0]
        try:
            pts = [[round(c[0]), round(c[1])] for c in coors]
        except:
            pts = [[round(c[0]), round(c[1])] for c in coors[0]]
        if len(pts) > 0:
            cv.fillPoly(
                classes_masks[get_mask_index(feat, classes)],
                [np.array(pts)],
                1
            )  # fill with ones if cell present

    mask = np.concatenate(classes_masks, axis=2)
    return mask

In [ ]:
def get_metrics(df, ground_truth_path, predicted_path, classes, shape, name):
    data = {
        'name': name
    }
    gj = geojson.load(open(ground_truth_path))
    ground_truth = get_mask(shape, gj['features'], classes)
    
    gj = geojson.load(open(predicted_path))
    predicted = get_mask(shape, gj['features'], classes)
    #predicted = postprocess_mask(predicted, True)
    #predicted = merge_nearest_endocards(predicted)
    
    for c_idx, c in enumerate(classes):
        data[f'IoU {c}'] = IoU(ground_truth[:, :, c_idx], predicted[:, :, c_idx])
        data[f'Dice {c}'] = dice_coef(ground_truth[:, :, c_idx], predicted[:, :, c_idx])

    
    return pd.concat([
            df,
            pd.DataFrame([data])
        ])

In [ ]:
def calculate_metrics():
    df = pd.DataFrame()
    
    ground_truth_path = r'E:\Master Thesis\DP3 results\Doctors\SRel - opravene'
    predicted_path = r'E:\Master Thesis\DP3 results\Doctors\SRel - endocard'
    classes = ['Endocarium']
    
    json_path = './data/images_512.json'
    
    with open(json_path) as json_file:
        json_data = json.load(json_file)
    
    ground_truth_files = glob.glob(f'{ground_truth_path}\\*.geojson')
    predicted_files = glob.glob(f'{predicted_path}\\*.geojson')
    
    for image in tqdm(json_data['images'], total=len(json_data['images'])):
        name = image['name'].replace('HE', '')
        print(name)
        predicted = [geo for geo in predicted_files if name in geo]
        if len(predicted) == 0:
            continue
        predicted = predicted[0]
        
        ground_truth = [geo for geo in ground_truth_files if name in geo]
        if len(ground_truth) == 0:
            continue
            
        ground_truth = ground_truth[0]
        shape = (image['height'], image['width'])
    
        df = get_metrics(df, ground_truth, predicted, classes, shape, name)
    
    return df

In [ ]:
df = calculate_metrics()

In [ ]:
df

In [ ]:
df.mean()

In [ ]:
metrics_df = pd.DataFrame([
    {
        'Trieda': 'Endokard',
        'IoU [%]': df[df['IoU Endocarium'] != 0]['IoU Endocarium'].mean() * 100,
        'Min IoU [%]': df[df['IoU Endocarium'] != 0]['IoU Endocarium'].min() * 100,
        'Max IoU [%]': df[df['IoU Endocarium'] != 0]['IoU Endocarium'].max() * 100,
        'Dice [%]': df[df['Dice Endocarium'] != 0]['Dice Endocarium'].mean() * 100,
        'Min Dice [%]': df[df['Dice Endocarium'] != 0]['Dice Endocarium'].min() * 100,
        'Max Dice [%]': df[df['Dice Endocarium'] != 0]['Dice Endocarium'].max() * 100,
    },
])

In [ ]:
print(metrics_df.round(2).to_latex(index=False)) 